### Домашнее задание к лекции "Основы веб-скрапинга и работы с API"¶
А.Петряшев (DS-28)

Задание 1.
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

In [1]:
import requests

In [2]:
from bs4 import BeautifulSoup

In [3]:
import pandas as pd

In [6]:
url_s = 'https://habr.com/ru/all/'
KEYWORDS = ['python', 'парсинг']
all_links = []
pages = 10
for i in range(1, pages):
    habr = requests.get(url_s, "html.parser") 
    soup = BeautifulSoup (habr.text)
    next_page = soup.find('a', class_='arrows-pagination__item-link arrows-pagination__item-link_next').get('href')
    url_s = 'https://habr.com' + next_page
    posts = soup.find_all('article', class_='post post_preview')

    for post in posts:
        preview_list = post.text.lower()
        for key in KEYWORDS:
            if key in preview_list:
                preview = post.find('a', class_='post__title_link')
                preview_list = preview.text.lower()
                post_time = post.find('span', class_='post__time').text
                refs = preview.attrs.get('href')
                all_links.append(refs)
                
print(all_links)

['https://habr.com/ru/company/macloud/blog/560406/', 'https://habr.com/ru/company/domclick/blog/560300/', 'https://habr.com/ru/post/560338/', 'https://habr.com/ru/post/560078/', 'https://habr.com/ru/company/kts/blog/560058/', 'https://habr.com/ru/company/intel/blog/558224/', 'https://habr.com/ru/post/560118/', 'https://habr.com/ru/post/559560/', 'https://habr.com/ru/post/560072/', 'https://habr.com/ru/post/559796/']


In [7]:
habr_posts = pd.DataFrame()
for link in all_links:
    sup = BeautifulSoup(requests.get(link).text, 'html.parser')
    
    date = pd.to_datetime(sup.find('span', class_='post__time').get('data-time_published'), dayfirst=True).date()
   
    title = sup.find('span', class_='post__title-text').text
    
    row = {'date': date, 'title': title, 'link': link}
   
    habr_posts = pd.concat([habr_posts, pd.DataFrame([row])])  
habr_posts

,date,title,link
0,2021-06-01,Как учить протоколы без чтения RFC: как сэконо...,https://habr.com/ru/company/macloud/blog/560406/
0,2021-06-01,Генераторы для самых маленьких,https://habr.com/ru/company/domclick/blog/560300/
0,2021-05-31,Конечные автоматы и django,https://habr.com/ru/post/560338/
0,2021-05-31,Инструменты для алготрейдинга на Python. SMA +...,https://habr.com/ru/post/560078/
0,2021-05-31,Первые шаги в aiohttp,https://habr.com/ru/company/kts/blog/560058/
0,2021-05-31,"Разбираемся, как подавить шум в речи с помощью...",https://habr.com/ru/company/intel/blog/558224/
0,2021-05-31,Анализ результатов работы архитектуры YoloV3 н...,https://habr.com/ru/post/560118/
0,2021-05-31,"Domain-driven design, Hexagonal architecture o...",https://habr.com/ru/post/559560/
0,2021-05-30,Сокрытые драгоценности Python,https://habr.com/ru/post/560072/
0,2021-05-30,Прогнозирование временных рядов с помощью AutoML,https://habr.com/ru/post/559796/


Задание 2.
Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [10]:
import json

In [11]:
from datetime import datetime

In [82]:
EMAIL = ["xxx@x.ru", "yyy@y.com"]

hak_url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'


In [85]:
#avast_i = pd.DataFrame()
for emails in EMAIL:
    payload = {"emailAddresses":[emails]}
    data = {'Vaar-Version': '0', 'Vaar-Header-App-Product-Name': 'hackcheck-web-avast', 'Vaar-Header-App-Build-Version': '1.0.0', 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36', "emailAddresses": emails} 
    email_checked = emails
    hak_res = requests.post(hak_url, json = payload, headers = data)
    #print(hak_res.status_code)
    
    
    r = hak_res.json()['breaches']
    avast_i = pd.DataFrame()
    for k, v in r.items():
        
        post = email_checked
        date = datetime.strptime(v["publishDate"], '%Y-%m-%dT%H:%M:%SZ')
        descriptions = v["description"]
        site = v['site']
        #print(date, site, post)
        rows_ = {'email': post, 'date': date, 'site': site, 'descriptions': descriptions}
        avast_i = pd.concat([avast_i, pd.DataFrame([rows_])])
avast_i

,email,date,site,descriptions
0,yyy@y.com,2020-01-03,azcentral.com,"At an unconfirmed date, online Arizona newspap..."
0,yyy@y.com,2020-05-28,wishbone.io,"In January 2020, the online poll website Wishb..."
0,yyy@y.com,2017-11-04,myheritage.com,"In October 2017, a customer database belonging..."
0,yyy@y.com,2021-02-11,forums.vkmonline.com,"At an unconfirmed date, the Russian-language m..."
0,yyy@y.com,2019-06-13,canva.com,"In May 2019, graphic-design site Canva's datab..."
0,yyy@y.com,2016-10-24,dropbox.com,Cloud storage company Dropbox suffered a major...
0,yyy@y.com,2016-10-21,linkedin.com,"In 2012, online professional networking platfo..."
0,yyy@y.com,2017-03-01,rayli.com.cn,"On an unconfirmed date, Chinese gossip site Ra..."
0,yyy@y.com,2019-10-17,zynga.com,"In September 2019, the game developer Zynga wa..."
0,yyy@y.com,2016-10-21,adobe.com,"In October of 2013, criminals penetrated Adobe..."
